In [6]:
!pip install tabulate
!pip install sentence_transformers

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable


In [25]:
import numpy as np
from sentence_transformers import SentenceTransformer
from tabulate import tabulate
from sklearn.preprocessing import normalize

In [8]:
# Load the model
model = SentenceTransformer("all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Here we can make lists for datasets we want to order
animals = [
    "kitten", "puppy", "rabbit", "panda", "hedgehog",
    "hamster", "dolphin", "horse", "penguin",
    "fox", "deer", "owl", "cow", "chicken", "goat",
    "pig", "parrot", "squirrel", "rat", "snake", "spider",
    "bat", "vulture", "shark", "cockroach", "maggot", "worm", "hyena",
]

In [29]:
# This combines multiple similar words into a single word anchor to remove noise
def encode_anchor(words):
    vecs = model.encode(words)
    vecs = normalize(vecs)
    mean_vec = np.mean(vecs, axis = 0)
    mean_vec = mean_vec / np.linalg.norm(mean_vec)
    return mean_vec

In [10]:
# This function does the projection
def proj_meas(v1, v2, v3):
    v = v2-v1
    w = v3-v1
    proj = np.dot(w,v)/np.dot(v,v)*v
    d = np.linalg.norm(w - proj)
    
    t = np.dot(w, v) / np.dot(v, v) # t is how far along on the spectrum something is, 0.0 for v1, 1.0 for v2.
    proj_point = v1 + t * v
    return d, proj_point, t

In [30]:
# Here I make the list and sort them based on the scale
def make_scale_list(words1, words2, word_list):
    scale_scores = []
    dist_scores = []
    vec1 = encode_anchor(words1)
    vec2 = encode_anchor(words2)
        
    for word in word_list:
        deter = model.encode(word)
        deter = deter / np.linalg.norm(deter)

        d, proj, t = proj_meas(vec1, vec2, deter)
        scale_scores.append(t)
        dist_scores.append(d)

    scores, words, dists = zip(*sorted(zip(scale_scores, word_list, dist_scores)))
    # normed_scores = (scores-min(scores))/(max(scores)-min(scores))
    normed_dists = 1-(dists-min(dists))/(max(dists)-min(dists)) # Normalized makes a bit more sense here
    # Build table
    table = []
    for word, score, dist, nor_dist in zip(words, scores, dists, normed_dists):
        table.append([word, f"{score:.3f}", f"{dist:.3f}", f"{nor_dist:.3f}"])

    headers = ["Word", "t (scale)", "Distance", "Normalized Distance"]
    print('From ', words1, ' to ', words2, ':')
    print(tabulate(table, headers=headers, tablefmt="fancy_grid"))
        
make_scale_list(["angry", "hostile", "aggressive", "mean", "irritable"], ["friendly", "nice", "gentle", "kind", "affectionate"], animals)

From  ['angry', 'hostile', 'aggressive', 'mean', 'irritable']  to  ['friendly', 'nice', 'gentle', 'kind', 'affectionate'] :
╒═══════════╤═════════════╤════════════╤═══════════════════════╕
│ Word      │   t (scale) │   Distance │   Normalized Distance │
╞═══════════╪═════════════╪════════════╪═══════════════════════╡
│ squirrel  │       0.409 │      1.06  │                 0.834 │
├───────────┼─────────────┼────────────┼───────────────────────┤
│ cockroach │       0.411 │      1.099 │                 0.438 │
├───────────┼─────────────┼────────────┼───────────────────────┤
│ maggot    │       0.412 │      1.097 │                 0.452 │
├───────────┼─────────────┼────────────┼───────────────────────┤
│ snake     │       0.422 │      1.077 │                 0.655 │
├───────────┼─────────────┼────────────┼───────────────────────┤
│ vulture   │       0.429 │      1.075 │                 0.681 │
├───────────┼─────────────┼────────────┼───────────────────────┤
│ rat       │       0.43  │    